In [ ]:
import os
import re

import pandas as pd
import numpy as np
from tkinter import StringVar
from tkinter import Tk, Label, Button, filedialog, Listbox, Text, Scrollbar, END, MULTIPLE, Toplevel
from tkinter import messagebox
from tkcalendar import Calendar
from tkinter.messagebox import showinfo
from collections import Counter, defaultdict

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10000)

menu_font_size = 20
submenu_font_size = 12
last_sorting_method = None

def calc_revenue(df, start_date=None, end_date=None, store_name=None):
    df.columns = [col.replace('\n', '') for col in df.columns]
    df = df[df['狀態'].apply(lambda x: x.split('\n')[0] not in ['取消訂單', '逾期未取件'])]
    df['訂購日期'] = pd.to_datetime(df['訂購日期'], format='%Y/%m/%d')
    
    if start_date and end_date:
        df = df[(df['訂購日期'] >= start_date) & (df['訂購日期'] <= end_date)]

    if store_name:
        df = df[df['賣場名稱'].isin(store_name)]
    
    revenue = df['小計(A)'].apply(lambda x: str(x).replace(',', '')).astype(int).sum()
    return revenue


def read_data(df_paths, skiprows=None):
    df_list = [pd.read_excel(df_path, skiprows=skiprows) for df_path in df_paths]
    return pd.concat(df_list, axis=0)


def select_files():
    file_paths = filedialog.askopenfilenames(title="選擇檔案", filetypes=[("Excel files", "*.xlsx *.xls")])
    if file_paths:
        file_listbox.delete(0, END)
        selected_files.clear()
        selected_files.extend(file_paths)

        for file in file_paths:
            file_listbox.insert(END, os.path.basename(file))


def calculate_revenue():
    if not selected_files:
        messagebox.showinfo("提示", "未選擇任何檔案")
        return
    
    df = read_data(selected_files, skiprows=2)
    revenue = calc_revenue(df=df, start_date=start_date, end_date=end_date, store_name=None)
    messagebox.showinfo("營收結果", f"總營收: {revenue}")


def load_stroke_data(filepath):
    # Load CSV and create a dictionary of character to stroke count
    df = pd.read_csv(filepath, skiprows=4)
    stroke_dict = dict(zip(df['Character'], df['Strokes']))
    return stroke_dict

def stroke_sort(input_list, stroke_dict):
    def char_key(char):
        # If the character is in stroke dictionary, return its stroke count and char for further sorting
        if char in stroke_dict:
            return (stroke_dict[char], char)
        # Non-Chinese characters are placed with a default stroke count of 0 for initial sorting
        else:
            return (0, char)

    def string_key(s):
        # Convert each string into a tuple of sorting keys for each character
        return [char_key(char) for char in s]

    # Sort the list based on generated keys
    sorted_list = sorted(input_list, key=string_key)
    return sorted_list


def calculate_buyer_data(df, sort_by=None):
    df_filtered = df.loc[df.iloc[:, 0].map(lambda x: x != '團名')]
    df_filtered.iloc[:, 4:] = df_filtered.iloc[:, 4:].replace({'\u3000': ' '}, regex=True)
    prices = df_filtered.iloc[:, 3]
    buyers = df_filtered.iloc[:, 4:]
    buyer_data = defaultdict(lambda: {'count': 0, 'price': 0})

    for i, row in buyers.iterrows():
        price = int(prices[i])
        for buyer in row.dropna():
            buyer_data[buyer]['price'] += price
            buyer_data[buyer]['count'] += 1
    df_val = [[f"{key} ({val['count']})", 1, val['price']] for key, val in buyer_data.items()]
    df_buyer = pd.DataFrame(df_val, columns=['＊規格', '＊數量', '＊價格'])

    # Sort by selected option
    if sort_by == "筆畫":
        stroke_dict = load_stroke_data('kangxi-strokecount/kangxi-strokecount.csv')
        sorted_specifications = stroke_sort(df_buyer['＊規格'].tolist(), stroke_dict)

        # Reindex the DataFrame based on the sorted order
        df_buyer = df_buyer.set_index('＊規格').loc[sorted_specifications].reset_index()
    elif sort_by == "金額":
        df_buyer = df_buyer.sort_values(by="＊價格", ascending=False)
    elif sort_by == "不排序":
        pass
    return df_buyer


def select_files():
    file_paths = filedialog.askopenfilenames(title="選擇檔案", filetypes=[("Excel files", "*.xlsx *.xls")])
    if file_paths:
        file_listbox.delete(0, END)
        selected_files.clear()
        selected_files.extend(file_paths)

        for file in file_paths:
            file_listbox.insert(END, os.path.basename(file))


def display_buyer_data(sort_by=None):
    if not selected_files:
        showinfo("提示", "未選擇任何檔案")
        return
    print(selected_files)
    df = read_data(selected_files)
    df_buyer = calculate_buyer_data(df, sort_by=sort_by)
    
    output_text.delete("1.0", END)
    output_text.insert(END, df_buyer.to_string(index=False))

# def save_batch_upload_file():
#     if not selected_files:
#         showinfo("提示", "未選擇任何檔案")
#         return

#     df = read_data(selected_files)
#     df_buyer = calculate_buyer_data(df)
    
#     save_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx *.xls")])
#     if save_path:
#         df_buyer.to_excel(save_path, index=False)
#         showinfo("成功", f"批次上架檔已儲存至 {save_path}")


def save_batch_upload_file():
    global last_sorting_method

    if last_sorting_method is None:
        showinfo("錯誤", "請先選擇排序方式！")
        return
    
    df = read_data(selected_files)
    df_buyer = calculate_buyer_data(df, sort_by=last_sorting_method)
    # Proceed with file generation using the last selected sorting method
    # Pass `last_sorting_method` to the generation function
    save_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx *.xls")])
    if save_path:
        df_buyer.to_excel(save_path, index=False)
        showinfo("成功", f"批次上架檔已儲存至 {save_path}")


def set_sorting_method(method):
    global last_sorting_method
    last_sorting_method = method
    display_buyer_data(sort_by=method)


def adjust_window_size():
    root.update_idletasks()  # 更新視窗的子元件
    width = root.winfo_reqwidth()   # 獲取視窗需求的寬度
    height = root.winfo_reqheight() # 獲取視窗需求的高度
    root.geometry(f"{width}x{height}")  # 根據內容設置視窗大小


def pick_date():
    def set_date():
        global start_date, end_date
        start_date = cal_start.get_date()
        end_date = cal_end.get_date()
        date_label.config(text=f"選擇範圍: {start_date} ~ {end_date}")
        date_picker.destroy()
    
    date_picker = Toplevel(root)
    date_picker.title("選擇日期範圍")
    
    Label(date_picker, text="開始日期:").pack()
    cal_start = Calendar(date_picker, selectmode='day', date_pattern='yyyy/mm/dd')
    cal_start.pack()
    
    Label(date_picker, text="結束日期:").pack()
    cal_end = Calendar(date_picker, selectmode='day', date_pattern='yyyy/mm/dd')
    cal_end.pack()
    
    Button(date_picker, text="確定", command=set_date).pack()


def revenue_menu():
    for widget in root.winfo_children():
        widget.destroy()
    
    Label(root, text="請選擇日期範圍:", font=('Arial', 12)).pack(pady=10)
    global date_label
    date_label = Label(root, text="尚未選擇日期", font=('Arial', 12))
    date_label.pack()
    Button(root, text="選擇日期", font=('Arial', 12), command=pick_date).pack(pady=5)
    
    Button(root, text="選擇檔案", font=('Arial', 12), command=select_files).pack(pady=10)
    Label(root, text="已選擇的檔案:", font=('Arial', 12)).pack()
    
    global file_listbox
    file_listbox = Listbox(root, width=50, height=8, font=('Arial', 12))
    file_listbox.pack()
    
    Button(root, text="開始計算", font=('Arial', 12), command=calculate_revenue).pack(pady=20)
    Button(root, text="返回主選單", font=('Arial', 12), command=root.quit).pack(pady=10)


def process_count_data(input_data):
    """
    處理數量統計輸入資料，解析並計算每個標題的數據總和。
    :param input_data: str, 輸入框中的多行數據
    :return: dict, 每個標題對應的數量總和
    """
    results = {}
    current_title = None

    # 遍歷每行輸入資料
    for line in input_data.strip().split('\n'):
        line = line.strip()  # 去除前後空格
        if not line:
            continue
        elif not re.search(r'\(.*?\)|（.*?）', line):
            current_title = line
            if current_title not in results:
                results[current_title] = 0
        else:
            # 如果是數據行，提取數量
            match = re.search(r'\((\d+)\)|（(\d+)）', line)
            if match and current_title:
                quantity = int(match.group(1) or match.group(2)) 
                results[current_title] += quantity

    return results


def calculate_counts():
    """
    讀取輸入框資料，處理並顯示計算結果。
    """
    input_data = input_text.get("1.0", END).strip()
    if not input_data:
        showinfo("提示", "請輸入數據！")
        return

    # 使用處理函式計算統計結果
    results = process_count_data(input_data)
    output = "\n".join([f"{title}: {count}" for title, count in results.items()])
    output_text.delete("1.0", END)
    output_text.insert(END, output)


def count_menu():
    for widget in root.winfo_children():
        widget.destroy()

    Label(root, text="數量統計工具", font=("Arial", submenu_font_size)).pack(pady=10)

    Label(root, text="請輸入資料:", font=("Arial", submenu_font_size)).pack(pady=5)

    global input_text, output_text
    input_text = Text(root, width=50, height=15, font=("Arial", submenu_font_size))
    input_text.pack(pady=5)

    Button(root, text="計算", font=("Arial", submenu_font_size), command=calculate_counts).pack(pady=10)

    Label(root, text="輸出結果:", font=("Arial", submenu_font_size)).pack(pady=5)

    output_text = Text(root, width=50, height=15, font=("Arial", submenu_font_size))
    #output_label = Label(root, textvariable=output_text, font=("Arial", submenu_font_size), justify="left")
    output_text.pack(pady=5)

    Button(root, text="返回主選單", font=("Arial", submenu_font_size), command=main_menu).pack(pady=10)

    adjust_window_size()


def batch_upload_menu():
    global last_sorting_method  # Access the global variable

    for widget in root.winfo_children():
        widget.destroy()

    Label(root, text="批次上架工具", font=('Arial', submenu_font_size)).pack(pady=10)
    Button(root, text="選擇檔案", font=('Arial', submenu_font_size), command=select_files).pack(pady=5)
    Label(root, text="已選擇的檔案:", font=('Arial', submenu_font_size)).pack()
    
    global file_listbox
    file_listbox = Listbox(root, width=50, height=5, font=('Arial', submenu_font_size))
    file_listbox.pack()

    Button(root, text="不排序", font=('Arial', submenu_font_size), command=lambda: set_sorting_method("不排序")).pack(pady=5)
    Button(root, text="按買家筆畫排序", font=('Arial', submenu_font_size), command=lambda: set_sorting_method("筆畫")).pack(pady=5)
    Button(root, text="按金額排序", font=('Arial', submenu_font_size), command=lambda: set_sorting_method("金額")).pack(pady=5)

    global output_text
    output_text = Text(root, width=60, height=15)
    output_text.pack(pady=10)
    
    Button(root, text="生成批次上架檔", font=('Arial', submenu_font_size), command=save_batch_upload_file).pack(pady=10)
    Button(root, text="返回主選單", font=('Arial', submenu_font_size), command=main_menu).pack(pady=10)
    adjust_window_size()


def buyer_member_menu():
    def convert_input():
        """將輸入的文字轉換為DataFrame並顯示"""
        raw_text = input_text.get("1.0", "end").strip()  # 獲取輸入框中的文字
        sections = [section.strip() for section in raw_text.split("---") if section.strip()]  # 分段處理並去掉多餘空格
        processed_data = []

        max_columns = 0
        for section in sections:
            rows = [row.strip() for row in section.split("\n") if row.strip()]
            processed_data.append(rows)
            max_columns = max(max_columns, len(rows))

        # 填充短於最大列數的行
        processed_data = [row + [""] * (max_columns - len(row)) for row in processed_data]

        # 轉成DataFrame
        df = pd.DataFrame(processed_data)
        output_text.delete("1.0", "end")
        output_text.insert("1.0", df.to_string(index=False, header=False))

        # 儲存處理後的DataFrame以便生成 CSV 使用
        buyer_member_menu.processed_df = df

    def save_to_csv():
        """將轉換後的DataFrame輸出為CSV檔案"""
        if not hasattr(buyer_member_menu, 'processed_df') or buyer_member_menu.processed_df.empty:
            showinfo("錯誤", "尚未轉換資料，請先輸入並點擊轉換按鈕。")
            return

        file_path = filedialog.asksaveasfilename(defaultextension=".csv",
                                                    filetypes=[("CSV files", "*.csv")],
                                                    title="保存檔案")
        if file_path:
            buyer_member_menu.processed_df.to_csv(file_path, index=False, header=False, encoding='utf-8-sig')
            showinfo("成功", f"已成功保存為 {file_path}")

    for widget in root.winfo_children():
        widget.destroy()

    Label(root, text="團員統計工具", font=("Arial", submenu_font_size)).pack(pady=10)

    Label(root, text="請輸入資料(以 --- 表示換行):", font=("Arial", submenu_font_size)).pack(pady=5)

    global input_text, output_text
    input_text = Text(root, width=50, height=15, font=("Arial", submenu_font_size))
    input_text.pack(pady=5)

    Button(root, text="轉換", font=("Arial", submenu_font_size), command=convert_input).pack(pady=10)

    Label(root, text="輸出結果:", font=("Arial", submenu_font_size)).pack(pady=5)

    # output_text = Text(root, width=50, height=15, font=("Arial", submenu_font_size))
    # #output_label = Label(root, textvariable=output_text, font=("Arial", submenu_font_size), justify="left")
    # output_text.pack(pady=5)

    # Button(root, text="返回主選單", font=("Arial", submenu_font_size), command=main_menu).pack(pady=10)

    output_text = Text(root, width=60, height=15)
    output_text.pack(pady=10)
    
    Button(root, text="生成團員統計表", font=('Arial', submenu_font_size), command=save_to_csv).pack(pady=10)
    Button(root, text="返回主選單", font=('Arial', submenu_font_size), command=main_menu).pack(pady=10)
    adjust_window_size()



def split_dataframe_by_header_adjusted(df, col_name="團名", header_value="團名"):
    header_indices = df[df[col_name] == header_value].index.tolist()
    header_indices.insert(0, -1)  
    header_indices.append(len(df))  
    
    result = []
    for i in range(len(header_indices) - 1):
        start_idx = header_indices[i]
        end_idx = header_indices[i + 1]
        sub_df = df.iloc[start_idx + 1 : end_idx].reset_index(drop=True)  # 切出範圍
        
        if i > 0:
            sub_df.columns = df.iloc[start_idx].values
        result.append(sub_df)
    
    return result

# def convert_to_member_summary(dfs, stroke_dict=None):
#     member_summary = defaultdict(lambda: defaultdict(list))  # 結構：{成員: {團: [商品(數量)]}}
#     seen_group_name = set()
#     group_name_list = []
#     for df in dfs:
#         if "團名" not in df.columns or df.empty:
#             continue
#         group_name = df.iloc[0, 0]  # 團名
#         for _, row in df.iterrows():
#             product = row["角色"]  # 商品名稱
#             for member in row[4:]:  # 從第4欄開始是成員名
#                 if pd.notna(member):  # 跳過空值
#                     member_summary[member][group_name].append(product)
#                     if group_name not in seen_group_name:
#                         group_name_list.append(group_name)
#                         seen_group_name.add(group_name)
#     summary_data = defaultdict(dict)
#     for member, groups in member_summary.items():
#         for group, products in groups.items():
#             # 維持原排序並計算商品數量
#             product_counts = []
#             seen_products = set()
#             for product in products:
#                 if product not in seen_products:
#                     count = products.count(product)
#                     product_counts.append(f"{product}({count})")
#                     seen_products.add(product)
#             summary_data[member][group] = "\n".join(product_counts)
            
#     # 使用 stroke_sort 對團員進行排序
#     stroke_dict = load_stroke_data('kangxi-strokecount/kangxi-strokecount.csv')
#     # sorted_specifications = stroke_sort(df_buyer['＊規格'].tolist(), stroke_dict)

#     # # Reindex the DataFrame based on the sorted order
#     # df_buyer = df_buyer.set_index('＊規格').loc[sorted_specifications].reset_index()
#     if stroke_dict is not None:
#         sorted_members = stroke_sort(list(summary_data.keys()), stroke_dict)
#     else:
#         sorted_members = list(summary_data.keys())
    
#     # 重新整理結果，根據排序後的團員順序輸出
#     sorted_summary_data = {member: summary_data[member] for member in sorted_members}
    
#     return pd.DataFrame(sorted_summary_data).T.fillna("").loc[:, group_name_list]  # 轉置，填補空值為空字串

def convert_to_member_summary(dfs, stroke_dict=None):
    member_summary = defaultdict(lambda: defaultdict(list))  # 結構：{成員: {團: [(商品, 單價)]}}
    seen_group_name = set()
    group_name_list = []

    for df in dfs:
        if "團名" not in df.columns or df.empty:
            continue
        group_name = df.iloc[0, 0]  # 團名

        for _, row in df.iterrows():
            product = row["角色"]  # 商品名稱
            unit_price = int(row.iloc[3])  # 單價（確保是整數）

            for member in row[4:]:  # 從第5欄開始是成員名
                if pd.notna(member):  # 跳過空值
                    member_summary[member][group_name].append((product, unit_price))
                    if group_name not in seen_group_name:
                        group_name_list.append(group_name)
                        seen_group_name.add(group_name)

    summary_data = defaultdict(dict)
    total_revenue = {}  # 存儲每個成員的總金額

    for member, groups in member_summary.items():
        total_price_all_groups = 0
        for group, products in groups.items():
            product_counts = []
            seen_products = {}

            for product, unit_price in products:
                if product not in seen_products:
                    count = sum(1 for p, _ in products if p == product)  # 計算商品出現次數
                    total_price = count * unit_price  # 計算總金額
                    product_counts.append(f"{product}({count})${total_price}")  # 金額為整數
                    seen_products[product] = count
                    total_price_all_groups += total_price  # 累加到總金額
            
            summary_data[member][group] = "\n".join(product_counts)
        
        total_revenue[member] = total_price_all_groups  # 記錄該成員的總金額
    
    # 使用 stroke_sort 對團員進行排序
    stroke_dict = load_stroke_data('kangxi-strokecount/kangxi-strokecount.csv')
    if stroke_dict is not None:
        sorted_members = stroke_sort(list(summary_data.keys()), stroke_dict)
    else:
        sorted_members = list(summary_data.keys())

    # 重新整理結果，根據排序後的團員順序輸出
    sorted_summary_data = {member: summary_data[member] for member in sorted_members}

    # 轉換成 DataFrame，填補空值為空字串
    result_df = pd.DataFrame(sorted_summary_data).T.fillna("").loc[:, group_name_list]

    # 加入「總金額」欄位，確保它是最後一欄
    result_df["總金額"] = [total_revenue[member] for member in sorted_members]

    return result_df




def member_goods_menu():
    for widget in root.winfo_children():
        widget.destroy()

    Label(root, text="團員商品統計工具", font=('Arial', submenu_font_size)).pack(pady=10)
    Button(root, text="選擇檔案", font=('Arial', submenu_font_size), command=select_files).pack(pady=5)
    Label(root, text="已選擇的檔案:", font=('Arial', submenu_font_size)).pack()

    global file_listbox
    file_listbox = Listbox(root, width=50, height=5, font=('Arial', submenu_font_size))
    file_listbox.pack()

    global output_text
    output_text = Text(root, width=60, height=15)
    output_text.pack(pady=10)

    Button(root, text="生成團員商品統計", font=('Arial', submenu_font_size), command=generate_member_goods_summary).pack(pady=10)
    Button(root, text="返回主選單", font=('Arial', submenu_font_size), command=main_menu).pack(pady=10)
    adjust_window_size()


def generate_member_goods_summary():
    if not selected_files:
        showinfo("提示", "未選擇任何檔案")
        return
    
    df = read_data(selected_files)
    split_dfs = split_dataframe_by_header_adjusted(df, col_name="團名", header_value="團名")
    summary_df = convert_to_member_summary(split_dfs)
    
    output_text.delete("1.0", END)
    output_text.insert(END, summary_df.to_string(index=True))  # 顯示統計結果

    save_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx *.xls")])
    if save_path:
        summary_df.to_excel(save_path, index=True)
        showinfo("成功", f"團員商品統計已儲存至 {save_path}")


def main_menu():
    for widget in root.winfo_children():
        widget.destroy()
    
    Label(root, text="請選擇功能", font=("Arial", menu_font_size)).pack(pady=20)
    Button(root, text="計算營收", font=("Arial", menu_font_size), command=revenue_menu).pack(pady=20)
    Button(root, text="數量統計", font=("Arial", menu_font_size), command=count_menu).pack(pady=20)
    Button(root, text="批次上架", font=("Arial", menu_font_size), command=batch_upload_menu).pack(pady=20)
    Button(root, text="團員統計", font=("Arial", menu_font_size), command=buyer_member_menu).pack(pady=20)
    Button(root, text="團員商品", font=("Arial", menu_font_size), command=member_goods_menu).pack(pady=20)
    
root = Tk()
root.title("營收與買家統計工具")
selected_files = []

main_menu()

root.update_idletasks()
root.minsize(root.winfo_reqwidth(), root.winfo_reqheight())
root.geometry('300x200+100+100')
root.mainloop()


In [ ]:
import pandas as pd
from collections import defaultdict
# 假設你提供的 DataFrame 形式

def split_dataframe_by_header_adjusted(df, col_name="團名", header_value="團名"):
    header_indices = df[df[col_name] == header_value].index.tolist()
    header_indices.insert(0, -1)  
    header_indices.append(len(df))  
    
    result = []
    for i in range(len(header_indices) - 1):
        start_idx = header_indices[i]
        end_idx = header_indices[i + 1]
        sub_df = df.iloc[start_idx + 1 : end_idx].reset_index(drop=True)  # 切出範圍
        
        if i > 0:
            sub_df.columns = df.iloc[start_idx].values
        result.append(sub_df)
    
    return result

def convert_to_member_summary(dfs):
    member_summary = defaultdict(lambda: defaultdict(list))  # 結構：{成員: {團: [商品(數量)]}}
    
    for df in dfs:
        if "團名" not in df.columns or df.empty:
            continue
        group_name = df.iloc[0, 0]  # 團名
        for _, row in df.iterrows():
            product = row["角色"]  # 商品名稱
            for member in row[4:]:  # 從第5欄開始是成員名
                if pd.notna(member):  # 跳過空值
                    member_summary[member][group_name].append(product)
    
    summary_data = defaultdict(dict)
    for member, groups in member_summary.items():
        for group, products in groups.items():
            product_counts = {p: products.count(p) for p in set(products)}  # 計算每商品數量
            summary_data[member][group] = "\n".join(f"{p}({c})" for p, c in product_counts.items())
    
    return pd.DataFrame(summary_data).T.fillna("")  # 轉置，填補空值為空字串

# 呼叫函數
split_dfs = split_dataframe_by_header_adjusted(df)
convert_to_member_summary(split_dfs)

,ind20(Roka),ind22(veru)
黎卓,真理小卡(1)\n景元小卡(1)\n星期日小卡(1)\n丹恆小卡(1)\n刃小卡(1)\n應...,心海小卡(5)
慕容染,真理小卡(1)\n景元小卡(1),
哆啦ともえ,景元小卡(1),
藍因,應星小卡(1)\n真理小卡(1)\n景元小卡(1)\n穹幼年小卡(1)\n星期日小卡(1)\...,
陳宇芝,砂金小卡(1)\n景元小卡(1),
廖卉姍,應星小卡(1)\n真理小卡(1)\n景元小卡(2)\n星期日小卡(1)\n丹恆小卡(1)\n...,
繾綣,應星小卡(1)\n真理小卡(2)\n景元小卡(1)\n星期日小卡(2)\n丹恆小卡(1)\n...,
Chun Hu,景元小卡(2)\n星期日小卡(1)\n丹恆小卡(1)\n刃小卡(1)\n砂金小卡(2)\n穹...,
Ina Hazel,刃小卡(1)\n星期日小卡(1)\n丹恆小卡(1),
夏蔚凜,刃小卡(1)\n應星小卡(1)\n刃幼年小卡(2),


In [41]:
import pandas as pd
from collections import defaultdict

# 分割函數：處理原始資料，將團分成多個子 DataFrame
def split_dataframe_by_header_adjusted(df, col_name="團名", header_value="團名"):
    header_indices = df[df[col_name] == header_value].index.tolist()
    header_indices.insert(0, -1)  # 最初的 DataFrame 作為特殊段
    header_indices.append(len(df))  # 加入結尾作為分割點
    
    result = []
    for i in range(len(header_indices) - 1):
        start_idx = header_indices[i]
        end_idx = header_indices[i + 1]
        sub_df = df.iloc[start_idx + 1 : end_idx].reset_index(drop=True)
        
        # 若不是第一段，設定欄位名稱
        if i > 0:
            sub_df.columns = df.iloc[start_idx].values
        result.append(sub_df)
    return result

# 轉換函數：從分割的團資料，生成成員購買統計
def convert_to_member_summary(dfs):
    member_summary = defaultdict(lambda: defaultdict(list))  # 結構：{成員: {團: [商品(數量)]}}
    
    for df in dfs:
        if "團名" not in df.columns or df.empty:
            continue
        group_name = df.iloc[0, 0]  # 團名
        for _, row in df.iterrows():
            product = row["角色"]  # 商品名稱
            for member in row[4:]:  # 從第5欄開始是成員名
                if pd.notna(member):  # 跳過空值
                    member_summary[member][group_name].append(product)
    
    # 將統計轉換為 DataFrame
    summary_data = defaultdict(dict)
    for member, groups in member_summary.items():
        for group, products in groups.items():
            product_counts = {p: products.count(p) for p in set(products)}  # 計算每商品數量
            summary_data[member][group] = "\n".join(f"{p}({c})" for p, c in product_counts.items())
    
    return pd.DataFrame(summary_data).T.fillna("")  # 轉置，填補空值為空字串

# 範例資料
data = {
    "團名": [
        "團名", "ind20(Roka)", None, None, "團名", "ind22(veru)", None, None
    ],
    "支出/數量": [None, 5513, None, None, None, 1388, None, None],
    "角色": [
        None, "景元小卡", "刃小卡", "丹恆小卡", None, "原神小卡一套", "芙寧娜小卡", "心海小卡"
    ],
    "價格": [None, 75, 75, 75, None, 185, 55, 55],
    "1": ["黎卓", "黎卓", "黎卓", "黎卓", "白月曦", "Yu Yu", None, None],
    "2": ["慕容染", "Ina Hazel", "林芸瑄", "藍因", "錢米拉", "Li Yun Xuan", None, None],
    "3": ["哆啦???", "廖卉姍", "藍因", "Ina Hazel", None, "陳冠伶", None, None],
    "4": ["藍因", "繾綣", "廖卉姍", "廖卉姍", None, "陳冠伶", None, None],
    "5": ["陳宇芝", "Chun Hu", "繾綣", "繾綣", None, None, None, None],
    "6": [None, "夏蔚凜", "Chun Hu", "Chun Hu", None, None, None, None],
}
df = pd.DataFrame(data)

# 主程式：切割並轉換
dfs = split_dataframe_by_header_adjusted(df)
# member_summary_df = convert_to_member_summary(dfs)

# # 輸出結果
print(dfs[2])

            團名     NaN    None    NaN    白月曦          錢米拉  None  None  None  \
0  ind22(veru)  1388.0  原神小卡一套  185.0  Yu Yu  Li Yun Xuan   陳冠伶   陳冠伶  None   
1         None     NaN   芙寧娜小卡   55.0   None         None  None  None  None   
2         None     NaN    心海小卡   55.0   None         None  None  None  None   

   None  
0  None  
1  None  
2  None  


In [ ]:
# check 


from collections import defaultdict
import pandas as pd

df = pd.read_excel('../data/1108(14團)_P.xlsx')

def load_stroke_data(filepath):
    # Load CSV and create a dictionary of character to stroke count
    df = pd.read_csv(filepath, skiprows=4)
    stroke_dict = dict(zip(df['Character'], df['Strokes']))
    return stroke_dict

def stroke_sort(input_list, stroke_dict):
    def char_key(char):
        # If the character is in stroke dictionary, return its stroke count and char for further sorting
        if char in stroke_dict:
            return (stroke_dict[char], char)
        # Non-Chinese characters are placed with a default stroke count of 0 for initial sorting
        else:
            return (0, char)

    def string_key(s):
        # Convert each string into a tuple of sorting keys for each character
        return [char_key(char) for char in s]

    # Sort the list based on generated keys
    sorted_list = sorted(input_list, key=string_key)
    return sorted_list

# Example usage:

def calculate_buyer_data(df, sort_by=None):
    df_filtered = df.loc[df.iloc[:, 0].map(lambda x: x != '下架日/開售日')]
    df_filtered.iloc[:, 4:] = df_filtered.iloc[:, 4:].replace({'\u3000': ' '}, regex=True)
    prices = df_filtered.iloc[:, 3]
    buyers = df_filtered.iloc[:, 4:]
    buyer_data = defaultdict(lambda: {'count': 0, 'price': 0})

    for i, row in buyers.iterrows():
        price = int(prices[i])
        for buyer in row.dropna():
            buyer_data[buyer]['price'] += price
            buyer_data[buyer]['count'] += 1
    df_val = [[f"{key} ({val['count']})", 1, val['price']] for key, val in buyer_data.items()]
    df_buyer = pd.DataFrame(df_val, columns=['＊規格', '＊數量', '＊價格'])

    # Sort by selected option
    if sort_by == "筆畫":
        stroke_dict = load_stroke_data('kangxi-strokecount/kangxi-strokecount.csv')
        sorted_specifications = stroke_sort(df_buyer['＊規格'].tolist(), stroke_dict)

        # Reindex the DataFrame based on the sorted order
        df_buyer = df_buyer.set_index('＊規格').loc[sorted_specifications].reset_index()
    elif sort_by == "金額":
        df_buyer = df_buyer.sort_values(by="＊價格", ascending=False)

    return df_buyer


# if __name__ == '__main__':
#     buyer_data = calculate_buyer_data(df, sort_by="筆畫").iloc[:, 0].apply(lambda x: x.split(' (')[0]).to_list()
#     find_similar_pairs(buyer_data, 2)


In [ ]:
import pandas as pd
import Levenshtein
from pprint import pprint
# 讀取筆畫數數據並轉為字典
stroke_data = pd.read_csv('kangxi-strokecount/kangxi-strokecount.csv', skiprows=4)
stroke_dict = dict(zip(stroke_data['Character'], stroke_data['Strokes']))

def get_stroke_count(char):
    # 返回字元的筆畫數，如果不在字典中，返回較高的值
    return stroke_dict.get(char, 0)

def calculate_string_stroke_count(s):
    return sum(get_stroke_count(char) for char in s)

def find_similar_pairs(strings, threshold_distance, threshold_strokes=0):
    similar_pairs = []
    stroke_counts = {s: calculate_string_stroke_count(s) for s in strings}

    for i in range(len(strings)):
        for j in range(i + 1, len(strings)):
            str1, str2 = strings[i], strings[j]
            
            # 筆畫數過濾
            stroke_diff = abs(stroke_counts[str1] - stroke_counts[str2])
            if stroke_diff > threshold_strokes:
                continue  # 如果筆畫數差距過大，跳過這對字串

            # Levenshtein 距離過濾
            distance = Levenshtein.distance(str1, str2)
            if distance <= threshold_distance:
                similar_pairs.append((str1, str2))

    return similar_pairs

df = pd.read_excel('../data/1108(14團)_1_P.xlsx')
buyer_data = calculate_buyer_data(df, sort_by="筆畫").iloc[:, 0].apply(lambda x: x.split(' (')[0]).to_list()
print('規則一、距離2以內 筆畫相同(抓符號或空格錯誤)')
rule1 = find_similar_pairs(buyer_data, threshold_distance=2, threshold_strokes=0)
#pprint(rule1)
print('規則二、距離1以內 不限筆畫差(抓中文錯字，假設只會錯一個字，抓不到錯兩個字以上)')
rule2 = find_similar_pairs(buyer_data, threshold_distance=1, threshold_strokes=999)
#pprint(rule2)
pd.set_option('display.max_rows', None)
pd.concat([pd.DataFrame({'規則一': rule1}), pd.DataFrame({'規則二': rule2})], axis=1)

規則一、距離2以內 筆畫相同(抓符號或空格錯誤)
規則二、距離1以內 不限筆畫差(抓中文錯字，假設只會錯一個字，抓不到錯兩個字以上)


,規則一,規則二
0,"(A, さと)","(Ciao Yu, Qiao Yu)"
1,"(Ciao Yu, Qiao Yu)","(王孟潔, 王筱潔)"
2,"(Dola, Moly)","(白玖, 白凌)"
3,"(Lin Yu, Lin Yuan)","(白玖, 白飯)"
4,"(Lin Yu, Tin Yui)","(白玖, 余玖)"
5,"(Mu Ku, Yu Yu)","(白凌, 白飯)"
6,"(Xiao Xiao, Xiao Xie)","(伊絲, 伊凜)"
7,"(Yi Ning, Yi Sin)","(安南, 安琪)"
8,"(ア草, 周月)","(吳宣, 吳雙)"
9,"(予夏, 神父)","(李珮綺, 江珮綺)"
